## Datos climaticos historicos del SMN  
Se toman de la estación meteorologica del Aerodromo de Bahia Blanca que se encuentra a 30 Km del PE La Castellana

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## meteostat
from datetime import datetime, timedelta
from meteostat import Hourly
from meteostat import Point, Daily
from meteostat import Stations

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get nearby weather stations
stations = Stations()

#Aerodromo de Bahia Blanca a 30 Km de La Castellana I
station_ba = stations.nearby(-38.7333, -62.1667) 
station_ba = station_ba.fetch(1)

# Print DataFrame
print(station_ba)

                         name country region    wmo  icao  latitude  \
id                                                                    
87750  Bahia Blanca Aerodrome      AR      B  87750  SAZB  -38.7333   

       longitude  elevation                        timezone hourly_start  \
id                                                                         
87750   -62.1667       75.0  America/Argentina/Buenos_Aires   1939-04-10   

      hourly_end daily_start  daily_end monthly_start monthly_end  distance  
id                                                                           
87750 2024-05-30  1956-02-02 2024-12-30    1956-01-01  2021-01-01       0.0  


### Obtención de datos historicos

In [ ]:
# Set time period Mask
start = datetime(2019, 1, 2)
#end = datetime(2024, 3, 31)


# Calcula end_date como la fecha actual para recuperar datos historicos del pronostico
current_date = datetime.now().date()
end_date = current_date #- timedelta(days=1)

# Convierte a datetime con la hora establecida a medianoche + 2hs asi al corregir a GMT-3 queda el dia completo
end = datetime.combine(end_date, datetime.min.time()) + timedelta(hours=2)

print("Start date:", start)
print("End date:", end)

In [ ]:
# Create Point for Bahia Blanca
station_ba = 87750  

# Get daily data Bahia Blanca
df_histo_meteostat = Hourly(station_ba, start, end)
#df_histo_meteostat = df_histo_meteostat.normalize()
df_histo_meteostat = df_histo_meteostat.fetch()

#Wind speed esta en Km/h lo convierto a m/s
df_histo_meteostat['wspd'] = (df_histo_meteostat['wspd'] / 3.6).round(1)

# Time Zone Correction Subtract 3 hours from the DateTimeIndex - UTC Buenos Aires
df_histo_meteostat.index = df_histo_meteostat.index - pd.to_timedelta(3, unit='h')

df_histo_meteostat = df_histo_meteostat[[ 'wspd', 'wdir', 'temp','pres','rhum']]

df_histo_meteostat.tail()

In [ ]:
df_histo_meteostat.index.name = 'FechaHora'

In [ ]:
df_histo_meteostat.describe()

In [ ]:
df_histo_meteostat.reset_index(inplace=True)

In [ ]:
#Se exporta el archivo datos historicos en un csv
df_histo_meteostat.to_csv('D:\\Documents\\MMA\\1.0 Tesis\\Datos\\METEOSTAT\\df_histo_meteostat.csv', sep=';', index=False, decimal=',')

### Obtención de datos de pronósticos

In [2]:
# Calcula end_date como la fecha actual + 10 días para incluir datos del pronostico
current_date = datetime.now().date()
end_date = current_date + timedelta(days=10)

# Set time period Mask
#start = current_date

# Convierte a datetime con la hora establecida a medianoche
start = datetime.combine(current_date, datetime.min.time()) + timedelta(hours=3)
end = datetime.combine(end_date, datetime.min.time()) + timedelta(hours=3)

print("Start date:", start)
print("End date:", end)

Start date: 2024-06-17 03:00:00
End date: 2024-06-27 03:00:00


In [3]:
# Create Point for Bahia Blanca
station_ba = 87750  

# Get daily data Bahia Blanca
df_fc_meteostat = Hourly(station_ba, start, end)
df_fc_meteostat = df_fc_meteostat.fetch()

#Wind speed esta en Km/h lo convierto a m/s
df_fc_meteostat['wspd'] = (df_fc_meteostat['wspd'] / 3.6).round(1)

# Time Zone Correction Subtract 3 hours from the DateTimeIndex - UTC Buenos Aires
df_fc_meteostat.index = df_fc_meteostat.index - pd.to_timedelta(3, unit='h')

df_fc_meteostat = df_fc_meteostat[[ 'wspd', 'wdir', 'temp','pres','rhum']]

df_fc_meteostat.tail()

,wspd,wdir,temp,pres,rhum
time,,,,,
2024-06-26 15:00:00,6.2,329.0,12.9,1017.3,61.0
2024-06-26 16:00:00,5.7,336.0,12.3,1017.1,65.0
2024-06-26 17:00:00,4.6,5.0,11.5,1017.1,69.0
2024-06-26 18:00:00,4.1,15.0,10.6,1017.2,74.0
2024-06-26 19:00:00,4.1,5.0,9.2,1017.0,80.0


In [4]:
df_fc_meteostat.index.name = 'FechaHora'

In [5]:
df_fc_meteostat.describe()

,wspd,wdir,temp,pres,rhum
count,236.000000,236.000000,236.000000,236.000000,236.000000
mean,4.259746,248.330508,7.334322,1015.806780,83.415254
std,1.340130,104.786880,3.652683,5.080019,13.214323
min,2.600000,2.000000,0.400000,1005.000000,47.000000
25%,3.100000,191.750000,4.650000,1012.650000,73.750000
50%,4.100000,297.000000,7.050000,1016.850000,85.000000
75%,5.100000,337.000000,9.600000,1020.400000,95.000000
max,9.800000,360.000000,16.900000,1022.800000,100.000000


In [6]:
df_fc_meteostat.reset_index(inplace=True)

In [8]:
# Find the minimum 'FechaHora'
min_fecha_hora = df_fc_meteostat['FechaHora'].min()

# Convert the minimum 'FechaHora' to string (date only)
min_fecha_hora_date_str = min_fecha_hora.strftime('%Y-%m-%d')

# Define the file path with the date string
file_path = f'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\METEOSTAT\\ds_fc_smn_{min_fecha_hora_date_str}.csv'

# Save the DataFrame to a CSV file with the corrected file path
df_fc_meteostat.to_csv(file_path, sep=';', index=False, decimal=',')